In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/2020-03-13/preprocessed.csv", index_col=0)

In [3]:
df

,sha,title,abstract_tokens
2,210a892deb1c61577f6fba58505fd65356ce6636,Incubation Period and Other Epidemiological Ch...,"['The', 'geographic', 'spread', 'of', '2019', ..."
3,e3b40cc8e0e137c416b4a2273a4dca94ae8178cc,Characteristics of and Public Health Responses...,"['In', 'December', '2019', ',', 'cases', 'of',..."
4,e9239100c5493ea914dc23c3d7a262f4326022ac,Distinct Roles for Sialoside and Protein Recep...,"['Coronaviruses', '(', 'CoVs', ')', 'are', 'co..."
5,469ed0f00c09e2637351c9735c306f27acf3aace,First two months of the 2019 Coronavirus Disea...,"['Similar', 'to', 'outbreaks', 'of', 'many', '..."
6,4e550e034ccca6fa2a91e481ddba24db67bc9ae5,Effectiveness of airport screening at detectin...,"['We', 'simulated', '100', '2019-nCoV', 'infec..."
...,...,...,...
9006,b9fe40055b01585084db291401c57f26d25fe478,Modelling the effective reproduction number of...,"['The', 'burden', 'of', 'vector-borne', 'disea..."
9007,edee452881f826fb72c58ee68a982789b12aa99d,Demographic Variations of MERS-CoV Infection a...,"['Introduction', '.', 'Middle', 'East', 'respi..."
9008,7fb385f3c0c5ce04f1c3d3c4f20678c0ae3a7031,"Development and validation of knowledge, attit...","['BACKGROUND', ':', 'Hajj', 'pilgrimage', 'fac..."
9009,e5a10a7f2c9bb07f74fcce076a62f63f859cf3a4,Global Critical Care: Moving Forward in Resour...,"['Caring', 'for', 'critically', 'ill', 'patien..."


In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')

In [5]:
import faiss
encoded_data = model.encode(df.abstract_tokens.tolist()[0:10])
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, len(encoded_data))))
faiss.write_index(index, 'paper.index')

In [6]:
def fetch_paper_info(dataframe_idx):
    info = df.iloc[dataframe_idx]
    meta_dict = dict()
    meta_dict['title'] = info['title']
    # meta_dict['Plot'] = info['Plot'][:500]
    return meta_dict
    
def search(query, top_k, index, model):
    t = time.time()
    query_vector = model.encode([query])
    top_k = index.search(query_vector, top_k)
    print('>>>> Results in Total Time: {}'.format(time.time()-t))
    top_k_ids = top_k[1].tolist()[0]
    top_k_ids = list(np.unique(top_k_ids))
    results =  [fetch_paper_info(idx) for idx in top_k_ids]
    return results

In [ ]:
import time
from pprint import pprint
query = "What is the incubation time of covid?"
results = search(query, top_k=5, index=index, model=model)
print("\n")
for result in results:
    print('\t', result)